In [5]:
import pandas as pd       
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
# from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import nltk
import yfinance as yf

In [6]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/cka/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
merc_df = pd.read_csv('./data/merc_news.csv')
merc_df['date'] = pd.to_datetime(merc_df['date'])
merc_df.set_index('date', inplace=True)
merc_df.sort_index(inplace=True)
merc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 492 entries, 2009-08-17 12:14:00+00:00 to 2020-06-11 14:22:00+00:00
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   headline   492 non-null    object
 1   sentiment  492 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB


In [15]:
merc = yf.Ticker('MERC')
merc_stock_df = merc.history(period='15y')
merc_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3776 entries, 2009-05-18 00:00:00-04:00 to 2024-05-16 00:00:00-04:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3776 non-null   float64
 1   High          3776 non-null   float64
 2   Low           3776 non-null   float64
 3   Close         3776 non-null   float64
 4   Volume        3776 non-null   int64  
 5   Dividends     3776 non-null   float64
 6   Stock Splits  3776 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 236.0 KB


In [16]:
# Define Merck text.
text = 'Merck (NYSE: MRK), known as MSD outside of the United States and Canada, today announced the completion of the acquisition of Harpoon Therapeutics, Inc. (Nasdaq: HARP). Harpoon is now a wholly-owned subsidiary of Merck, and Harpoon’s common stock will no longer be publicly traded or listed on the Nasdaq Stock Market. Harpoon’s lead candidate, MK-6070 (formerly known as HPN328), is a T-cell engager targeting delta-like ligand 3 (DLL3), an inhibitory canonical Notch ligand that is expressed at high levels in small cell lung cancer (SCLC) and neuroendocrine tumors. The safety, tolerability and pharmacokinetics of MK-6070 is currently being evaluated as monotherapy in a Phase 1/2 clinical trial (NCT04471727) in certain patients with advanced cancers associated with expression of DLL3. The study is also evaluating MK-6070 in combination with atezolizumab in certain patients with SCLC. In March 2022, the U.S. Food and Drug Administration (FDA) granted Orphan Drug Designation to MK-6070 for the treatment of SCLC.'
print(text)

Merck (NYSE: MRK), known as MSD outside of the United States and Canada, today announced the completion of the acquisition of Harpoon Therapeutics, Inc. (Nasdaq: HARP). Harpoon is now a wholly-owned subsidiary of Merck, and Harpoon’s common stock will no longer be publicly traded or listed on the Nasdaq Stock Market. Harpoon’s lead candidate, MK-6070 (formerly known as HPN328), is a T-cell engager targeting delta-like ligand 3 (DLL3), an inhibitory canonical Notch ligand that is expressed at high levels in small cell lung cancer (SCLC) and neuroendocrine tumors. The safety, tolerability and pharmacokinetics of MK-6070 is currently being evaluated as monotherapy in a Phase 1/2 clinical trial (NCT04471727) in certain patients with advanced cancers associated with expression of DLL3. The study is also evaluating MK-6070 in combination with atezolizumab in certain patients with SCLC. In March 2022, the U.S. Food and Drug Administration (FDA) granted Orphan Drug Designation to MK-6070 for t

In [17]:
merc_stock_df.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
merc_stock_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2009-05-18 00:00:00-04:00,0.796483,0.825446,0.774761,0.796483,60400
2009-05-19 00:00:00-04:00,0.825446,0.825446,0.753038,0.760279,89100
2009-05-20 00:00:00-04:00,0.796483,0.796483,0.709594,0.716834,262400
2009-05-21 00:00:00-04:00,0.709593,0.716834,0.615464,0.622704,118000
2009-05-22 00:00:00-04:00,0.622705,0.760279,0.622705,0.760279,251400


# Pre-Processing 

- Remove special characters
- Tokenizing
- Lemmatizing/Stemming
- Stop word removal

In [4]:
# sentence tokenizer  ## May affect NER
headlines = sent_tokenize(text.lower())

['merck (nyse: mrk), known as msd outside of the united states and canada, today announced the completion of the acquisition of harpoon therapeutics, inc. (nasdaq: harp).',
 'harpoon is now a wholly-owned subsidiary of merck, and harpoon’s common stock will no longer be publicly traded or listed on the nasdaq stock market.',
 'harpoon’s lead candidate, mk-6070 (formerly known as hpn328), is a t-cell engager targeting delta-like ligand 3 (dll3), an inhibitory canonical notch ligand that is expressed at high levels in small cell lung cancer (sclc) and neuroendocrine tumors.',
 'the safety, tolerability and pharmacokinetics of mk-6070 is currently being evaluated as monotherapy in a phase 1/2 clinical trial (nct04471727) in certain patients with advanced cancers associated with expression of dll3.',
 'the study is also evaluating mk-6070 in combination with atezolizumab in certain patients with sclc.',
 'in march 2022, the u.s. food and drug administration (fda) granted orphan drug design

In [6]:
# # word tokenizer 
# word_tokenize(text.lower())

## Lemmatizing & Stemming

**Lemmatizing** and **stemming** are two forms of shortening words so we can combine similar forms of the same word.

When we "**lemmatize**" data, we take words and attempt to return their *lemma*, or the base/dictionary form of a word.

In [7]:
# Instantiate RegExp Tokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')

# "Run" Tokenizer
text_tokens = tokenizer.tokenize(text.lower())

In [8]:
# Instantiate lemmatizer. 
lemmatizer = WordNetLemmatizer()

In [9]:
# Lemmatize tokens.
tokens_lem = [lemmatizer.lemmatize(i) for i in text_tokens]

In [11]:
# Compare tokens to lemmatized version.
# list(zip(text_tokens, tokens_lem))

# Print only those lemmatized tokens that are different.
[(text_tokens[i], tokens_lem[i]) for i in range(len(text_tokens)) if text_tokens[i] != tokens_lem[i]]

[('as', 'a'),
 ('states', 'state'),
 ('therapeutics', 'therapeutic'),
 ('as', 'a'),
 ('levels', 'level'),
 ('tumors', 'tumor'),
 ('as', 'a'),
 ('patients', 'patient'),
 ('cancers', 'cancer'),
 ('patients', 'patient')]

## Stop Word Removal

In [12]:
# Remove stopwords from "text_tokens."
no_stop_words = [token for token in text_tokens if token not in stopwords.words('english')]

In [13]:
# Check it
print(no_stop_words)

['merck', '(nyse:', 'mrk', '),', 'known', 'msd', 'outside', 'united', 'states', 'canada', ',', 'today', 'announced', 'completion', 'acquisition', 'harpoon', 'therapeutics', ',', 'inc', '.', '(nasdaq:', 'harp', ').', 'harpoon', 'wholly', '-owned', 'subsidiary', 'merck', ',', 'harpoon', '’s', 'common', 'stock', 'longer', 'publicly', 'traded', 'listed', 'nasdaq', 'stock', 'market', '.', 'harpoon', '’s', 'lead', 'candidate', ',', 'mk', '-6070', '(formerly', 'known', 'hpn328', '),', '-cell', 'engager', 'targeting', 'delta', '-like', 'ligand', '3', '(dll3),', 'inhibitory', 'canonical', 'notch', 'ligand', 'expressed', 'high', 'levels', 'small', 'cell', 'lung', 'cancer', '(sclc)', 'neuroendocrine', 'tumors', '.', 'safety', ',', 'tolerability', 'pharmacokinetics', 'mk', '-6070', 'currently', 'evaluated', 'monotherapy', 'phase', '1', '/2', 'clinical', 'trial', '(nct04471727)', 'certain', 'patients', 'advanced', 'cancers', 'associated', 'expression', 'dll3', '.', 'study', 'also', 'evaluating', 'm